In [ ]:
import numpy as np
from scipy.optimize import fsolve
from scipy.special import lpmv

In [ ]:
def pf(l): return np.sqrt((2*l + 1)/(4*np.pi))
def β(l): return (l - 1)*(2*l + 1)
def ω(l): return np.sqrt(l*(l - 1)*(l + 2))
def _Bo(r, ρ, g, σ): return ρ*g*r**2/σ 
def _Oh(r, ρ, μ, σ): return μ/np.sqrt(σ*ρ*r)
def _T(r, ρ, σ): return np.sqrt(ρ*r**3/σ)
def _U(r, ρ, σ): return np.sqrt(σ/(ρ*r))
def _Ω(f, T): return f*2.*np.pi*T
def _Γ(a, f, g): return a*(2.*np.pi*f)**2/g
def _freq(OMEGA, T_unit): return (OMEGA/(2.*np.pi*T_unit))
def _amp(GAMMA, grav, freq): return (GAMMA*grav/(2.*np.pi*freq)**2)
def s_pos(t, Bo, Γ, Ω, ϕ): return Bo*(Γ/Ω**2)*np.cos(Ω*t + ϕ*np.pi)
def s_vel(t, Bo, Γ, Ω, ϕ): return - Bo*(Γ/Ω)*np.sin(Ω*t + ϕ*np.pi)
def s_acc(t, Bo, Γ, Ω, ϕ): return - Bo*Γ*np.cos(Ω*t + ϕ*np.pi)
def d_lpmv(m, x): return (m + 1.)*(x*lpmv(0, m, x) - lpmv(0, m + 1, x))/(1. - x**2 + 1e-12)
def a_m(m, x): return - (2./3.)*((2*m + 1)*d_lpmv(m, x))/((m - 1.)*m*(m + 1.)*(m + 2))

In [ ]:
def model3(params):
    r, z0, v0, a, f, ϕ, tend, dt, C, D = params

    Bo = _Bo(r, ρ, g, σ)
    Oh = _Oh(r, ρ, μ, σ)
    T = _T(r, ρ, σ)
    U = _U(r, ρ, σ)
    Ω = _Ω(f, T)
    Γ = _Γ(a, f, g)

    dt /= T
    
    time = np.arange(0, tend/T, dt)
    
    Z = np.zeros(len(time))
    dZ = np.zeros(len(time))
    
    Z[0] = z0/r + s_pos(0, Bo, Γ, Ω, ϕ) - 1.
    dZ[0] = v0/U + s_vel(0, Bo, Γ, Ω, ϕ)
    
    A_2 = np.zeros(len(time))
    dA_2 = np.zeros(len(time))

    y = np.array([Z[0], dZ[0], A_2[0], dA_2[0]])
    
    def F_decoupled(y, y_prev, t, dt):
        z1, z2, a1, a2 = y
        z1_prev, z2_prev, a1_prev, a2_prev = y_prev

        g_prev = - Bo + s_acc(t - dt, Bo, Γ, Ω, ϕ)
        g_new = - Bo + s_acc(t, Bo, Γ, Ω, ϕ)

        h_prev = -2*Oh*β(2)*a2_prev - ω(2)**2*a1_prev
        h_new = -2*Oh*β(2)*a2 - ω(2)**2*a1

        F1 = z1 - z1_prev - dt * 0.5 * (z2 + z2_prev)
        F2 = z2 - z2_prev - dt * 0.5 * (g_new + g_prev)
        F3 = a1 - a1_prev - dt * 0.5 * (a2 + a2_prev)
        F4 = a2 - a2_prev - dt * 0.5 * (h_new + h_prev)
        
        return np.array([F1, F2, F3, F4])
        
    def F_coupled(y, y_prev, t, dt):
        z1, z2, a1, a2 = y
        z1_prev, z2_prev, a1_prev, a2_prev = y_prev

        g_prev = max(- 2.*β(2)*Oh*z2_prev - C*z1_prev - C*a1_prev, 0) - Bo + s_acc(t - dt, Bo, Γ, Ω, ϕ)
        g_new = max(- 2.*β(2)*Oh*z2 - C*z1 - C*a1, 0) - Bo + s_acc(t, Bo, Γ, Ω, ϕ)

        δ_prev = -(g_prev - s_acc(t - dt, Bo, Γ, Ω, ϕ) + Bo)
        δ_new = -(g_new - s_acc(t, Bo, Γ, Ω, ϕ) + Bo)
        
        h_prev = -2*Oh*β(2)*a2_prev - ω(2)**2*a1_prev +  D*δ_prev*(5/3)*3*(1 + δ_prev/3)
        h_new = -2*Oh*β(2)*a2 - ω(2)**2*a1 +  D*δ_new*(5/3)*3*(1 + δ_new/3)

        F1 = z1 - z1_prev - dt * 0.5 * (z2 + z2_prev)
        F2 = z2 - z2_prev - dt * 0.5 * (g_new + g_prev)
        F3 = a1 - a1_prev - dt * 0.5 * (a2 + a2_prev)
        F4 = a2 - a2_prev - dt * 0.5 * (h_new + h_prev)
        
        return np.array([F1, F2, F3, F4])
        
    t_bound = np.zeros(len(time))
    for i in range(1, len(time)):
        t = time[i-1]

        y_prev = y.copy()

        if y_prev[0] - y_prev[2] < 0:
            y = fsolve(F_coupled, y_prev, args=(y_prev, t, dt))
            t_bound[i-1] = 1 
        else:
            y = fsolve(F_decoupled, y_prev, args=(y_prev, t, dt))
            
        Z[i] = y[0]
        dZ[i] = y[1]
        A_2[i] = y[2]
        dA_2[i] = y[3]
        
    return time, Z + 1., Z + 1. - s_pos(time, Bo, Γ, Ω, ϕ), s_pos(time, Bo, Γ, Ω, ϕ), A_2,\
        T*time, r*(Z + 1.), r*(Z + 1. - s_pos(time, Bo, Γ, Ω, ϕ)), r*s_pos(time, Bo, Γ, Ω, ϕ), r*A_2, t_bound

In [ ]:
ρ = 0.95e3
μ = 20e-3
σ = 21e-3
g = 9.8067
r = 0.78 #mm
pos = 2.50 #mm
T_unit = _T(r*1e-3, ρ, σ)
C = 1.0
D = 1.0
pattern_OMEGA = r'OMEGA([\d.]+)'
pattern_GAMMA = r'GAMMA([\d.]+)'
pattern_radius = r'radius([\d.]+)'
pattern_freq = r'freq([\d.]+)'
pattern_amp = r'amp([\d.]+)'
pattern_pos = r'pos([\d.]+)'
pattern_phi = r'phi([\d.]+)'
dt_ratio = 1.0
dt = 5e-4
dt *= dt_ratio
tend = 1.0
cut = 0.6
cut2 = 1.0
tmp_time = np.arange(0, tend + dt, dt)
c_rom = int(cut*(len(tmp_time) - 1))
c2_rom = int(cut2*(len(tmp_time) - 1))
pos_ROM = {}

In [ ]:
freq_range = np.arange(0.7, 5.6, 0.1)
a_range = np.arange(0.1, 15.1, 0.1)
phi_range = np.arange(0, 2.0, 0.5)

for _phi in phi_range:
    for _OMEGA in freq_range:
        for _GAMMA in a_range:
            f = _freq(_OMEGA, T_unit)
            a =_amp(_GAMMA, g, f)
                
            T3, Z3, Z3_lab, S_3, A2_3, t3, z3, z3_lab, s_3, a2_3, t_bound = model3([r*1e-3, pos*1e-3, 0., a, f, _phi, tend, dt, C, D])

            mean_z3_lab = np.copy(z3_lab)
            mean_z3_lab -= np.mean(z3_lab[c_rom:c2_rom])

            amp_ROM = 2*np.sqrt(np.nansum(mean_z3_lab[c_rom:c2_rom]**2)*(tmp_time[1] - tmp_time[0])/(tmp_time[c2_rom] - tmp_time[c_rom]))*1e3
            airlayer_ROM = np.maximum(z3[c_rom:c2_rom] - (r*1e-3) - a2_3[c_rom:c2_rom]/2, 0)*1e3
            pos_ROM[_GAMMA, _OMEGA, _phi] = amp_ROM, np.mean(airlayer_ROM), np.max(airlayer_ROM)
            print(_OMEGA, _GAMMA, _phi, amp_ROM)

            